<img src="https://camo.githubusercontent.com/d7a1f81a2ee7576ab86720d9135ab3c915550e3945a7859f1c0300ab22ac1cec/687474703a2f2f707974686f6e2d76697375616c697a6174696f6e2e6769746875622e696f2f666f6c69756d2f5f696d616765732f666f6c69756d5f6c6f676f2e6a7067"> [Folium](https://github.com/python-visualization/folium) is built on the data wrangling strengths of the Python ecosystem and the mapping strengths of the Leaflet.js (JavaScript) library. Simply, manipulate your data in Python, then visualize it on a leaflet map via Folium. Folium makes it easy to visualize data that’s been manipulated in Python, on an interactive Leaflet map. This library has a number of built-in tilesets from `OpenStreetMap`, `Mapbox`, etc.

## Create a base map
First, we should learn how to find the longitude and latitude coordinates using Python.

<img src="https://geopy.readthedocs.io/en/stable/_images/logo-wide.png" width=200> [Geopy](https://github.com/geopy/geopy) is a python client for several popular geocoding web services. Geocoding is the process to identify geographic coordinates like latitude and longitude of a given city/country/address. This is very useful during marking position on the map in data visualization using Folium. First let's install `folium` and `geopy`.

In [ ]:
import folium
import geopy

In [ ]:
# !pip install folium # I don't think we need to pip install anymore

In [ ]:
# !pip install geopy # I don't think we need to pip install anymore

We use `geopy.exec`, `geocodertimedout`, `geolocators`, `geopy.geocoder` in the below code to fetch the coordinates for the cities we are interested in.

In [ ]:
# Import pandas package
import pandas as pd
import numpy as np

# Define a dictionary containing  data
data = {'City':['San Jose', 'San Francisco', 'Toronto', 'Paris']}

# Convert the dictionary into DataFrame
df = pd.DataFrame(data)

# Observe the result
df

,City
0,San Jose
1,San Francisco
2,Toronto
3,Paris


### Now Let’s find the latitude and longitude of the following regions.

First, create a function `findGeocode` that uses the `Nominatim` object to find the coordinates using the `geocode()` method. We will be using the `findGeocode` function to look for the map coordinates for all the states and cities.

In [ ]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

# Declare an empty list to store latitude and longitude of values
# of city column
longitude = []
latitude = []

# function to find the coordinate of a given city
def findGeocode(city):
    # try and catch is used to overcome the exception thrown by geolocator
    # using geocodertimedout
    try:
        # Specify the user_agent as your app name and it should not be none
        geolocator = Nominatim(user_agent="CS133")
        return geolocator.geocode(city)
    except GeocoderTimedOut:
        return None



In [ ]:
location_sf = findGeocode("San Francisco")
print(location_sf)
print(location_sf.latitude)
location_sf

San Francisco, California, United States
37.7792588


Location(San Francisco, California, United States, (37.7792588, -122.4193286, 0.0))

In [ ]:
findGeocode("Mount Whitney")

Location(Mount Whitney, Inyo County, California, United States, (36.5785905, -118.2921377, 0.0))

In [ ]:
test = findGeocode("60 S 9th St, San Jose, CA 95112")
test

Location(60, South 9th Street, Horace Mann, San Jose, Santa Clara County, California, 95112, United States, (37.33902371422434, -121.88136989105814, 0.0))

In [ ]:
test.altitude # Just FYI these all seem to be 0, so I wouldn't trust these altitude numbers

0.0

In [ ]:
test.address

'60, South 9th Street, Horace Mann, San Jose, Santa Clara County, California, 95112, United States'

In [ ]:
# street, city, county, state, country, or
# postalcode
findGeocode({"postalcode": 95112, "state": "California", "city": "San Jose", "street": "60 South 9th Street"})

Location(60, South 9th Street, Horace Mann, San Jose, Santa Clara County, California, 95112, United States, (37.33902371422434, -121.88136989105814, 0.0))

Now for each city, we call `findGeocode()` to look get the longitude and latitude coordinates.

In [ ]:
# each value from city column
# will be fetched and sent to
# function find_geocode
for i in (df["City"]):
    if findGeocode(i) != None:
        loc = findGeocode(i)
        # coordinates returned from function is stored into
        # two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)

    # if coordinate for a city not found, insert "NaN" indicating
    # missing value
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

# now add the Longitude and Latitude columns to dataframe
df["Longitude"] = longitude
df["Latitude"] = latitude

df

,City,Longitude,Latitude
0,San Jose,-121.890591,37.336166
1,San Francisco,-122.419329,37.779259
2,Toronto,-79.383935,43.653482
3,Paris,2.348391,48.853495


In [ ]:
findGeocode("USA")

Location(United States, (39.7837304, -100.445882, 0.0))

In [ ]:
# We shouldn't need this unless geopy times out

# df["Longitude"] = [-121.890591, -122.419906, -79.383935, 2.348391]
# df["Latitude"] = [37.336166, 37.779026, 43.653482, 48.853495]

# # City

# df

Here are the coordinates for San Jose

In [ ]:
sjla = df[(df.City=="San Jose")].Latitude.iloc[0] # adding iloc because of the future warning
sjlo = df[(df.City=="San Jose")].Longitude.iloc[0] # adding iloc because of the future warning
print(float(sjla), float(sjlo))

37.3361663 -121.890591


### Let's create a base map that centers on San Jose

In [ ]:
# import folium package
import folium

# Map method of folium return Map object

# Here we pass coordinates (latitude, longitude) of San Jose
# and starting Zoom level = 12

sj = folium.Map(location = [float(sjla), float(sjlo)],
                zoom_start = 8)
sj

## Add a pin-drop with pop-up text.

Let's manually get the coordinates for San Jose State University. Go to [Google map](https://www.google.com/maps), and search for San Jose State University. Click on the pin drop, right-click on the coordinates to copy. This is what I got `37.33544329470733, -121.88105004356032`.

In [ ]:
# Pass a string in popup parameter
sj = folium.Map(location = [float(sjla), float(sjlo)],
                zoom_start = 16)
# paste the coordinates for San Jose States to make the Pin Drop Marker
folium.Marker([37.33544329470733, -121.88105004356032],
               popup = 'My School').add_to(sj)

sj

## Add a line to the map
Let's create another pin drop at the California Academy of Sciences, and then we will draw a line between the two pin drops. From Google Map, I got the coordinate `37.77009356373172, -122.46604105704547` for the California Academy of Sciences.

In [ ]:
sj = folium.Map(location = [float(sjla), float(sjlo)],
                zoom_start = 9)
folium.Marker([37.33544329470733, -121.88105004356032],
               popup = 'SJSU').add_to(sj)
folium.Marker([37.77009356373172, -122.46604105704547],
              popup = 'California Academy of Sciences').add_to(sj)

# Add a line to the map by using line method. It connect both coordinates by the line

folium.PolyLine(locations = [(-87.359296,35.00118),(-85.606675,34.984749),[-85.431413,34.124869],[-85.184951,32.859696],[-85.069935,32.580372],[-84.960397,32.421541],[-85.004212,32.322956],[-84.889196,32.262709],[-85.058981,32.13674],[-85.053504,32.01077],[-85.141136,31.840985],[-85.042551,31.539753],[-85.113751,31.27686],[-85.004212,31.003013],[-85.497137,30.997536],[-87.600282,30.997536],[-87.633143,30.86609],[-87.408589,30.674397],[-87.446927,30.510088],[-87.37025,30.427934],[-87.518128,30.280057],[-87.655051,30.247195],[-87.90699,30.411504],[-87.934375,30.657966],[-88.011052,30.685351],[-88.10416,30.499135],[-88.137022,30.318396],[-88.394438,30.367688],[-88.471115,31.895754],[-88.241084,33.796253],[-88.098683,34.891641],[-88.202745,34.995703],[-87.359296,35.00118]],
                ).add_to(sj)

sj

### Icons and Custom Icons

In [ ]:
font_awesome_icon =folium.Icon(color="red",
                                icon="flag", # more icons here: https://fontawesome.com/v4/icons/
                                prefix='fa', # include this if using font awesome icons
                                )

folium.Marker(
      location=[37.27009356373172, -121.96604105704547],
      icon=font_awesome_icon,
      popup="Font Awesome Icon"
  ).add_to(sj)

sj

In [ ]:
# upload an image file to your drive and get the shareable link

# make a link that folium can use by using this structure
# "https://lh3.googleusercontent.com/d/FILEID"
# Example Sharable link:
# https://drive.google.com/file/d/1MorbW5mV-HqlqbOGOXCtxtNZaYMKqPV3/view?usp=sharing
# Example FILEID = 1MorbW5mV-HqlqbOGOXCtxtNZaYMKqPV3

icon_url = "https://lh3.googleusercontent.com/d/1MorbW5mV-HqlqbOGOXCtxtNZaYMKqPV3"

icon = folium.CustomIcon(
      icon_url, # make the icon url using the instructions above
      icon_size=(50, 50),
      icon_anchor=(25, 25),
      popup_anchor=(-25, -30))

folium.Marker(
      location=[36.77009356373172, -121.46604105704547],
      icon=icon,
      popup="My Custom Icon"
  ).add_to(sj)

sj

## Create a Choropleth map to show the Deaths per Million in each State in the US



### Choropleth map

Choropleth Maps display divided geographical areas or regions that are coloured, shaded or patterned in relation to a data variable. This provides a way to visualize values over a geographical area, which can show variation or patterns across the displayed location. To create the Choropleth maps, the first thing is to get the shapefile or geo json files you would like to apply in your personaliszd map and then merge your data accordingly.


### Let's use the data of the US Covid-19 cases (May 8, 2022) to visualize the trends with a choropleth map

In [ ]:
# Data collected from Johns Hopkins and US Census
jhu_data="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/05-08-2022.csv"
census="https://www2.census.gov/programs-surveys/popest/datasets/2020-2021/state/totals/NST-EST2021-alldata.csv"
# This is a geojson file that contains the polygon coordinates for the shapes of all states in the US
us_shape="https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"

In [ ]:
# Load the Population data from US Census
pop = pd.read_csv(census, encoding = "ISO-8859-1")
pop.rename(columns = {'STATE':'FIPS'}, inplace = True)
pop.head(8)

,SUMLEV,REGION,DIVISION,FIPS,NAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,NPOPCHG_2020,NPOPCHG_2021,...,NETMIG2020,NETMIG2021,RESIDUAL2020,RESIDUAL2021,RBIRTH2021,RDEATH2021,RNATURALINC2021,RINTERNATIONALMIG2021,RDOMESTICMIG2021,RNETMIG2021
0,10,0,0,0,United States,331449281,331501080,331893745,51799,392665,...,12247,244622,0,0,10.798957,10.352637,0.446319,0.737485,0.000000,0.737485
1,20,1,0,0,Northeast Region,57609148,57525633,57159838,-83515,-365795,...,-79429,-333592,-1024,-1151,9.950554,10.492070,-0.541516,0.977386,-6.794897,-5.817511
2,20,2,0,0,Midwest Region,68985454,68935174,68841444,-50280,-93730,...,-47266,-85268,-718,5156,10.860566,11.058248,-0.197682,0.549222,-1.786994,-1.237772
3,20,3,0,0,South Region,126266107,126409007,127225329,142900,816322,...,134454,770076,528,-3385,11.168835,10.777476,0.391359,0.886268,5.186064,6.072332
4,20,4,0,0,West Region,78588572,78631266,78667134,42694,35868,...,4488,-106594,1214,-620,10.767153,8.947910,1.819243,0.487570,-1.842880,-1.355309
5,40,3,6,1,Alabama,5024279,5024803,5039877,524,15074,...,3337,23380,-75,242,11.191613,12.890226,-1.698613,0.247201,4.398749,4.645950
6,40,4,9,2,Alaska,733391,732441,732673,-950,232,...,-2012,-3353,39,-54,12.667956,7.700425,4.967532,0.718033,-5.295151,-4.577118
7,40,4,8,4,Arizona,7151502,7177986,7276316,26484,98330,...,26791,97504,-3,-6,10.584669,10.469547,0.115121,0.619608,12.871739,13.491347


In [ ]:
# Load the Covid-19 data from Johns Hopkins
covid = pd.read_csv(jhu_data, encoding = "ISO-8859-1")
covid['FIPS'] = covid['FIPS'].astype(int)
covid.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,...,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,Date,People_Tested,Mortality_Rate
0,Alabama,US,2022-05-09 04:31:57,32.3182,-86.9023,1302397,19601,NaN,NaN,1,...,NaN,NaN,1.504994,84000001.0,USA,NaN,NaN,2022-05-08,NaN,NaN
1,Alaska,US,2022-05-09 04:31:57,61.3707,-152.4044,254466,1253,NaN,NaN,2,...,NaN,NaN,0.492404,84000002.0,USA,NaN,NaN,2022-05-08,NaN,NaN
2,American Samoa,US,2022-05-09 04:31:57,-14.2710,-170.1320,5923,28,NaN,NaN,60,...,NaN,NaN,0.472733,16.0,ASM,NaN,NaN,2022-05-08,NaN,NaN
3,Arizona,US,2022-05-09 04:31:57,33.7298,-111.4312,2025435,30189,NaN,NaN,4,...,19289892.0,NaN,1.490495,84000004.0,USA,265017.749694,NaN,2022-05-08,NaN,NaN
4,Arkansas,US,2022-05-09 04:31:57,34.9697,-92.3731,837154,11408,NaN,NaN,5,...,5126657.0,NaN,1.362712,84000005.0,USA,169880.383219,NaN,2022-05-08,NaN,NaN


In [ ]:
# Merge the Population and Covid-19 dataframes using State as the common column
us_covid = covid.merge(pop[['FIPS', 'POPESTIMATE2021']], on = ['FIPS'], how = "left")
us_covid = us_covid[['Province_State','Lat','Long_','Confirmed','Deaths','FIPS','POPESTIMATE2021']]
us_covid.rename(columns = {'POPESTIMATE2021':'Population', 'Province_State':'State'}, inplace = True)
us_covid.head()

,State,Lat,Long_,Confirmed,Deaths,FIPS,Population
0,Alabama,32.3182,-86.9023,1302397,19601,1,5039877.0
1,Alaska,61.3707,-152.4044,254466,1253,2,732673.0
2,American Samoa,-14.2710,-170.1320,5923,28,60,NaN
3,Arizona,33.7298,-111.4312,2025435,30189,4,7276316.0
4,Arkansas,34.9697,-92.3731,837154,11408,5,3025891.0


In [ ]:
# Calculate and rounding population to millions with 2 digits, and creating two new columns
us_covid['Population (million)'] = round((us_covid['Population']/1000000),2)
us_covid['Cases per Million'] = round((us_covid['Confirmed']/us_covid['Population (million)']),2)
us_covid['Deaths per Million'] = round((us_covid['Deaths']/us_covid['Population (million)']),2)

In [ ]:
# Drop rows with missing values
us_covid.dropna(inplace=True)
us_covid.head()

,State,Lat,Long_,Confirmed,Deaths,FIPS,Population,Population (million),Cases per Million,Deaths per Million
0,Alabama,32.3182,-86.9023,1302397,19601,1,5039877.0,5.04,258412.10,3889.09
1,Alaska,61.3707,-152.4044,254466,1253,2,732673.0,0.73,348583.56,1716.44
3,Arizona,33.7298,-111.4312,2025435,30189,4,7276316.0,7.28,278219.09,4146.84
4,Arkansas,34.9697,-92.3731,837154,11408,5,3025891.0,3.03,276288.45,3765.02
5,California,36.1162,-119.6816,9289178,90416,6,39237836.0,39.24,236727.27,2304.18


In [ ]:
us_covid = us_covid.sort_values(by = "Deaths per Million",
                                ascending = False).reset_index(drop=True)
us_covid.head(10).style.background_gradient(cmap='Reds',
                                subset = ['Deaths per Million'])

,State,Lat,Long_,Confirmed,Deaths,FIPS,Population,Population (million),Cases per Million,Deaths per Million
0,Mississippi,32.741600,-89.678700,798721,12454,28,2949965.000000,2.950000,270752.880000,4221.690000
1,Arizona,33.729800,-111.431200,2025435,30189,4,7276316.000000,7.280000,278219.090000,4146.840000
2,Oklahoma,35.565300,-96.928900,1042637,15940,40,3986639.000000,3.990000,261312.530000,3994.990000
3,Alabama,32.318200,-86.902300,1302397,19601,1,5039877.000000,5.040000,258412.100000,3889.090000
4,West Virginia,38.491200,-80.954500,502930,6880,54,1782959.000000,1.780000,282544.940000,3865.170000
5,Arkansas,34.969700,-92.373100,837154,11408,5,3025891.000000,3.030000,276288.450000,3765.020000
6,Tennessee,35.747800,-86.692300,2034555,26192,47,6975218.000000,6.980000,291483.520000,3752.440000
7,Louisiana,31.169500,-91.867800,1240538,17276,22,4624047.000000,4.620000,268514.720000,3739.390000
8,New Jersey,40.298900,-74.521000,2288950,33488,34,9267130.000000,9.270000,246920.170000,3612.510000
9,Michigan,43.326600,-84.536100,2444891,36064,26,10050811.000000,10.050000,243272.740000,3588.460000


In [ ]:
# Create a Choropleth map to show the Deaths per Million by State
bins = list(us_covid['Deaths per Million'].quantile([0, 0.25, 0.50, 0.75, 1]))
covidmap = folium.Map(location=[45, -95], zoom_start=5)
# covidmap = folium.Map(location=[45, -95], tiles="OpenStreetMap", zoom_start=5)
# covidmap = folium.Map(location=[45, -95], tiles="Stamen Toner", zoom_start=5)
# bins
choropleth = folium.Choropleth(
    geo_data=us_shape, #geojson file
    data=us_covid, # contains the numerical information that we want to visualize
    name='choropleth',
    columns=['State', 'Deaths per Million'], # first: join key for the geojson file, second: numerical values
    key_on='properties.name', # getting this information from the geo_data json file iteself
    fill_color= 'YlOrRd',
    fill_opacity=0.3,
    line_opacity=0.2,
    legend_name='Deaths per Million',
    bins = bins,
    reset = True
).add_to(covidmap)

style_function = "font-size: 15px; font-weight: bold"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'],style=style_function, labels=False)
)

covidmap

## Create a Bubble Map to visualize the fully-vaccinated rate in California by County
A bubble map uses circles of different size to represent a numeric value on a territory. It displays one bubble per geographic coordinate, or one bubble per region (in this case the bubble is usually displayed in the baricentre of the region).


### California Covid-19 vaccination status as of May 2022


In [ ]:
# Data sources
# California county shapes in geojson format
ca_shape = "https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/california-counties.geojson"
# California vaccination status
ca_vacc = "https://data.chhs.ca.gov/dataset/e283ee5a-cf18-4f20-a92c-ee94a2866ccd/resource/130d7ba2-b6eb-438d-a412-741bde207e1c/download/covid19vaccinesbycounty.csv"
# California population by county
cnty_pop = "https://www2.census.gov/programs-surveys/popest/datasets/2020-2021/counties/totals/co-est2021-alldata.csv"
# California county coordinates
cnty_coord = "https://raw.githubusercontent.com/csbfx/advpy122-data/master/ca-county-boundaries.csv"

# The source of US States and Counties Boundaries: https://public.opendatasoft.com/explore/dataset/us-county-boundaries

In [ ]:
# Load CA counties coordinates
coord = pd.read_csv(cnty_coord,sep=";")
coord = coord[['Geo Point', 'NAME']]
coord.rename(columns={'Geo Point':'Lat_Long', 'NAME':'County'}, inplace=True)
coord.head()

,Lat_Long,County
0,"35.3428473738,-118.729900424",Kern
1,"36.0753604529,-119.815502709",Kings
2,"36.6057058971,-121.074995529",San Benito
3,"36.7581801263,-119.649322632",Fresno
4,"40.0046824769,-120.838597596",Plumas


In [ ]:
# Load population data by counties
cpop = pd.read_csv(cnty_pop,  encoding = "ISO-8859-1")
capop = cpop[(cpop.STNAME=="California")]
capop = capop[['CTYNAME', 'POPESTIMATE2021']]
capop.rename(columns={'CTYNAME':'County', 'POPESTIMATE2021':'Population'}, inplace=True)
capop['County'] = capop.County.str.replace(" County","")
capop = capop[capop.County.str.contains("California")==False]
capop = capop.reset_index()
capop.head()

,index,County,Population
0,192,Alameda,1648556
1,193,Alpine,1235
2,194,Amador,41259
3,195,Butte,208309
4,196,Calaveras,46221


In [ ]:
# Load the California vaccination status data
vacc = pd.read_csv(ca_vacc)
vacc = vacc[vacc.county.str.contains("All CA")==False]
vdata = vacc[vacc['administered_date']=='2022-05-08'].groupby('county').max()
vdata.reset_index(inplace=True)
vdata.head()

,county,administered_date,total_doses,cumulative_total_doses,partially_vaccinated,total_partially_vaccinated,fully_vaccinated,cumulative_fully_vaccinated,at_least_one_dose,cumulative_at_least_one_dose,california_flag,up_to_date_count,cumulative_up_to_date_count
0,Alameda,2022-05-08,696,3718080,42,83052,31,1367968,42,1451119,California,0,26
1,Alpine,2022-05-08,0,1458,0,63,0,530,0,593,NaN,0,0
2,Amador,2022-05-08,2,57346,0,1535,0,21581,0,23116,California,0,0
3,Butte,2022-05-08,39,305582,4,8785,3,117189,4,125980,California,0,11
4,Calaveras,2022-05-08,23,64417,2,1616,1,23947,2,25563,California,0,3


In [ ]:
vdata.rename(columns={'county':'County'}, inplace=True)
vdata.head()

,County,administered_date,total_doses,cumulative_total_doses,partially_vaccinated,total_partially_vaccinated,fully_vaccinated,cumulative_fully_vaccinated,at_least_one_dose,cumulative_at_least_one_dose,california_flag,up_to_date_count,cumulative_up_to_date_count
0,Alameda,2022-05-08,696,3718080,42,83052,31,1367968,42,1451119,California,0,26
1,Alpine,2022-05-08,0,1458,0,63,0,530,0,593,NaN,0,0
2,Amador,2022-05-08,2,57346,0,1535,0,21581,0,23116,California,0,0
3,Butte,2022-05-08,39,305582,4,8785,3,117189,4,125980,California,0,11
4,Calaveras,2022-05-08,23,64417,2,1616,1,23947,2,25563,California,0,3


In [ ]:
# Merge the above three dataframes using County as the common column
ca_vacc = vdata.merge(coord, on=['County'], how = "left")
ca_vacc_pop = ca_vacc.merge(capop[['County','Population']], on=['County'], how="left")
ca_vacc_pop.head()

,County,administered_date,total_doses,cumulative_total_doses,partially_vaccinated,total_partially_vaccinated,fully_vaccinated,cumulative_fully_vaccinated,at_least_one_dose,cumulative_at_least_one_dose,california_flag,up_to_date_count,cumulative_up_to_date_count,Lat_Long,Population
0,Alameda,2022-05-08,696,3718080,42,83052,31,1367968,42,1451119,California,0,26,"37.6504930265,-121.917998966",1648556.0
1,Alpine,2022-05-08,0,1458,0,63,0,530,0,593,NaN,0,0,"38.5972506332,-119.82065303",1235.0
2,Amador,2022-05-08,2,57346,0,1535,0,21581,0,23116,California,0,0,"38.446389504,-120.651115621",41259.0
3,Butte,2022-05-08,39,305582,4,8785,3,117189,4,125980,California,0,11,"39.6669293633,-121.600671735",208309.0
4,Calaveras,2022-05-08,23,64417,2,1616,1,23947,2,25563,California,0,3,"38.204606803,-120.554132098",46221.0


In [ ]:
# Remove rows that don't have longitude and latitude information
ca_vacc_pop = ca_vacc_pop[ca_vacc_pop.Lat_Long.notnull()]
# Add a new column for Fully vaccinated rate
ca_vacc_pop['Fully vaccinated rate'] = ca_vacc_pop['cumulative_fully_vaccinated']/ca_vacc_pop['Population']

<ipython-input-70-6a95fb4e2c4d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ca_vacc_pop['Fully vaccinated rate'] = ca_vacc_pop['cumulative_fully_vaccinated']/ca_vacc_pop['Population']


In [ ]:
loc = findGeocode("California")
ca_lat = loc.latitude
ca_long = loc.longitude
ca_lat, ca_long

(36.7014631, -118.755997)

In [ ]:
# How many counties in the merged dataframe
len(ca_vacc_pop)

58

In [ ]:
# Create a bubble map to show the vaccination rate by county in California
vaccmap = folium.Map(location=[ca_lat, ca_long], zoom_start=6)
for i in range(0,len(ca_vacc_pop)):
   fully_vacc_rate = ca_vacc_pop.iloc[i]['Fully vaccinated rate']
   lat_long = str(ca_vacc_pop.iloc[i]['Lat_Long']).split(',')
   lat = float(lat_long[0])
   long_ = float(lat_long[1])
   folium.Circle(
      location=[lat, long_],
      popup=ca_vacc_pop.iloc[i]['County']+', '+str(round(fully_vacc_rate*100,2))+'%',
      radius=fully_vacc_rate*35000,
      color='purple',
      fill=True,
      fill_color='purple'
   ).add_to(vaccmap)

# Show the map again
vaccmap

## Create a Choropleth Map to visualize the fully-vaccinated rate in California by County

In [ ]:
# Create a Choropleth map to show the vaccination rates by county in California
bins = list(ca_vacc_pop['Fully vaccinated rate'].quantile([0, 0.25, 0.50, 0.75, 1]))
vaccmap2 = folium.Map(location=[ca_lat, ca_long], zoom_start=6)

choropleth = folium.Choropleth(
    geo_data=ca_shape,
    data=ca_vacc_pop,
    name='choropleth',
    columns=['County', 'Fully vaccinated rate'],
    key_on='properties.name',
    fill_color= 'BuPu',
    fill_opacity=0.4,
    line_opacity=0.6,
    legend_name='Fully vaccinated rate',
    bins = bins,
    reset = True
).add_to(vaccmap2)

style_function = "font-size: 15px; font-weight: bold"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'],style=style_function, labels=False)
)

vaccmap2

# Geopandas - bonus cool stuff

In [ ]:
import geopandas as gpd

In [ ]:
housing_developments = gpd.read_file("/content/Affordable_Rental_Housing.geojson")
housing_developments

map_center = (housing_developments.LATITUDE.mean(), housing_developments.LONGITUDE.mean())
map_center

(37.31981939461864, -121.8806582173305)

In [ ]:
housing_developments

,OBJECTID,FACILITYID,INTID,DEVELOPMENTNAME,WEBSITE,DEVELOPER,DEVTYPE,PROPERTYMANAGER,PHONE,TENURETYPE,...,INCLUSIONARY,BONDTAXCREDITONLY,APN,LATITUDE,LONGITUDE,DATASOURCE,CREATIONDATE,LASTUPDATE,NOTES,geometry
0,8760,2032,2032,Raintree,None,"Grand Resource Consulting-Los Angeles, LLC",For-Profit,"Triumph Residential Services, Inc",(408) 296-7578,Rental,...,No,Yes,27914036,37.306930,-121.949019,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None,POINT (-121.94903 37.30693)
1,8761,2120,2120,Roosevelt Park Apts,https://www.firstcommunityhousing.org/property...,First Community Housing,Non-Profit,John Stewart Company,None,Rental,...,None,None,46712001,37.346323,-121.871349,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None,POINT (-121.87136 37.34633)
2,8762,2113,2113,Leigh Avenue Senior Apartments,https://www.firstcommunityhousing.org/property...,First Community Housing,Non-Profit,John Stewart Company,No phone,Rental,...,No,No,28432014,37.306606,-121.921173,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None,POINT (-121.92119 37.30661)
3,8763,2094,2094,Tamien Station Affordable,None,UrbanCo Tamien LLC,For-Profit,John Stewart Company,None,Rental,...,None,No,43413041,37.314229,-121.884881,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None,POINT (-121.88489 37.31423)
4,8764,2035,2035,Rose Garden Seniors,http://www.roemcorp.com/projects/rose-gardens/,ROEM Development Corporation,For-Profit,"FPI Management, Inc.",(408) 928-2750,Rental,...,No,No,48421029,37.364530,-121.828163,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None,POINT (-121.82818 37.36453)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,8991,1978,1978,Cypress Gardens Senior,https://jscosccha.com/property/cypress-gardens/,SCCHA,Non-Profit,John Stewart Company,(408) 248-1442,Rental,...,No,No,30330041,37.320655,-121.961106,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None,POINT (-121.96112 37.32066)
232,8992,1987,1987,La Fenetre,http://www.caremgt.com/la-fenetre.html,JSM Enterprises,For-Profit,CA Real Estate Management Corp.,(408)Â 295-2066,Rental,...,No,No,26406094,37.316227,-121.908726,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None,POINT (-121.90874 37.31623)
233,8993,2075,2075,Village Park Apartments,http://www.sheaapartments.com/apartments/ascent/,Shea Homes,For-Profit,Shea Properties,(844) 861-5236,Rental,...,Yes,None,70604013,37.245262,-121.802571,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None,POINT (-121.80258 37.24527)
234,8994,2042,2042,Sienna Senior,http://www.caremgt.com/sienna.html,JSM Enterprises,For-Profit,CA Real Estate Management Corp.,(408) 971-9640,Rental,...,No,No,43421037,37.311529,-121.877766,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None,POINT (-121.87778 37.31153)


In [ ]:
sj_neighborhoods = gpd.read_file("/content/Neighborhoods.geojson")
sj_neighborhoods

,OBJECTID,FACILITYID,INTID,NAME,SOURCE,LASTUPDATE,NOTES,ENTERPRISEID,SHAPE_Length,SHAPE_Area,geometry
0,1,1,1,Canoas Garden and Almaden,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000001,22588.968774,1.428200e+07,"POLYGON ((-121.87346 37.29531, -121.87363 37.2..."
1,2,2,2,Gramercy and Alexander,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000002,10362.371809,5.324318e+06,"POLYGON ((-121.83865 37.36076, -121.83814 37.3..."
2,3,3,3,Farm and Rubino Dr,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000003,16203.758969,9.343714e+06,"POLYGON ((-121.87093 37.28040, -121.87095 37.2..."
3,4,4,4,Mayfair,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000004,16325.544276,1.625017e+07,"POLYGON ((-121.84130 37.35957, -121.84133 37.3..."
4,5,5,5,Owsley,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000005,10550.867305,5.381624e+06,"POLYGON ((-121.84687 37.32381, -121.84675 37.3..."
...,...,...,...,...,...,...,...,...,...,...,...
292,293,293,293,Coyote Creek,None,2022-05-26 14:56:44+00:00,None,REF-NEIG-0000000293,122193.391489,6.207776e+08,"POLYGON ((-121.66734 37.16728, -121.66764 37.1..."
293,294,294,294,Rancho Santa Teresa Mobile Homes,None,2022-05-26 14:56:44+00:00,None,REF-NEIG-0000000294,9031.791734,3.988641e+06,"MULTIPOLYGON (((-121.81791 37.26481, -121.8190..."
294,295,295,295,Magic Sands Mobile Homes,None,2022-05-26 14:56:44+00:00,None,REF-NEIG-0000000295,9281.380616,4.940265e+06,"POLYGON ((-121.80288 37.25537, -121.80295 37.2..."
295,296,296,296,Westfield Oakridge,None,2022-05-26 14:56:44+00:00,None,REF-NEIG-0000000296,11380.908540,6.216530e+06,"POLYGON ((-121.86653 37.25033, -121.86664 37.2..."


In [ ]:
# https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html
# Predicates values intersects, contains, within, touches, crosses, overlaps
# Specifics about what the different predicate values are here:
# https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates

joined = sj_neighborhoods.sjoin(housing_developments, how="left", predicate="contains")
# This joined df has all of the neighborhoods present. Some
joined

,OBJECTID_left,FACILITYID_left,INTID_left,NAME,SOURCE,LASTUPDATE_left,NOTES_left,ENTERPRISEID,SHAPE_Length,SHAPE_Area,...,HUDUNITS,INCLUSIONARY,BONDTAXCREDITONLY,APN,LATITUDE,LONGITUDE,DATASOURCE,CREATIONDATE,LASTUPDATE_right,NOTES_right
0,1,1,1,Canoas Garden and Almaden,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000001,22588.968774,1.428200e+07,...,0.0,No,No,45518083,37.292926,-121.875116,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None
0,1,1,1,Canoas Garden and Almaden,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000001,22588.968774,1.428200e+07,...,0.0,None,None,45531053,37.295518,-121.875476,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None
0,1,1,1,Canoas Garden and Almaden,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000001,22588.968774,1.428200e+07,...,0.0,No,No,45609015,37.297282,-121.875966,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None
0,1,1,1,Canoas Garden and Almaden,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000001,22588.968774,1.428200e+07,...,0.0,No,No,45520068,37.299379,-121.876101,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None
0,1,1,1,Canoas Garden and Almaden,Census Blockgroup,2022-05-26 14:56:41+00:00,None,REF-NEIG-0000000001,22588.968774,1.428200e+07,...,148.0,No,None,45533006,37.285726,-121.871367,CSJ Housing,2023-11-28 22:23:59+00:00,2023-11-28 22:23:59+00:00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,293,293,293,Coyote Creek,None,2022-05-26 14:56:44+00:00,None,REF-NEIG-0000000293,122193.391489,6.207776e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
293,294,294,294,Rancho Santa Teresa Mobile Homes,None,2022-05-26 14:56:44+00:00,None,REF-NEIG-0000000294,9031.791734,3.988641e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
294,295,295,295,Magic Sands Mobile Homes,None,2022-05-26 14:56:44+00:00,None,REF-NEIG-0000000295,9281.380616,4.940265e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
295,296,296,296,Westfield Oakridge,None,2022-05-26 14:56:44+00:00,None,REF-NEIG-0000000296,11380.908540,6.216530e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN


In [ ]:
import numpy as np


def counting_developments(df):
  developer_counts = df.DEVELOPER.count()
  development_name_counts = df.DEVELOPMENTNAME.count()
  return max(developer_counts, development_name_counts)



# upcoming_development_counts = joined.groupby("NAME")[["DEVELOPMENTNAME", "DEVELOPER"]].size()
upcoming_development_counts = joined.groupby("NAME").apply(counting_developments)
upcoming_development_counts = upcoming_development_counts.replace(0, np.nan)

In [ ]:
upcoming_development_counts

NAME
Aborn and Silver Creek             NaN
Airport                            NaN
Alameda                            6.0
Alexian and Jackson                1.0
Alma                               3.0
                                  ... 
Winchester                         NaN
Winchester Lynnhaven               NaN
Windmill Springs                   NaN
Wooster and East Ct and West Ct    2.0
YB Lucretia                        2.0
Length: 295, dtype: float64

In [ ]:
neighborhood_choropleth = folium.Choropleth(geo_data="/content/Neighborhoods.geojson",
                                            highlight=True,
                                            overlay=True,
                                            data=upcoming_development_counts,
                                            name="San Jose Neighborhoods",
                                            # key_on="feature.properties.NAME", # both work
                                            key_on="properties.NAME", # both work
                                            legend_name="Number of Upcoming Housing Developments",
                                            # bins=[1,3,5,7,9, 11, 13],
                                            nan_fill_opacity=0,
                                            fill_color='YlGnBu'
                                            )
style_function = "font-size: 15px; font-weight: bold"

# Adding the neighborhood name
# Copied previous code, updated the variable "choropleth" to my "neighborhood_choropleth" variable name
# Then updated ["name"] to ["NAME"] to match the information in the geojson file
neighborhood_choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['NAME'],style=style_function, labels=False)
)

In [ ]:
housing_map = folium.Map(map_center)

neighborhood_choropleth.add_to(housing_map)
housing_map